In [1]:
# Python script to plot figures in an ordered way
# by Alvaro Sanchez-Monge

#
#-----------------------------------------------------------------------
# Import required packages
#
import os
import sys
import numpy as np
import argparse
import astropy
from astropy.io import fits
from astropy import wcs
from astropy import units as u
from astropy.coordinates import Angle
from matplotlib import rc
from matplotlib import cm
from matplotlib.patches import Rectangle
from matplotlib.patches import Ellipse
from matplotlib.patches import ConnectionPatch
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt


from matplotlib_scalebar.scalebar import ScaleBar
import aplpy

import matplotlib
matplotlib.rcParams.update({'font.size': 22})

In [2]:
# Load all FITS files
#Dense
hdu_image_11 = fits.open('../../../data/Herschel_ngc6334/Doris/NGC6334_Coldens_r18p2_Galactic_delhead.fits')[0]


#temp
hdu_image_21 = fits.open('../../../data/Herschel_ngc6334/Doris/NGC6334_Dust_Temp_r36p3_Galactic_delhead.fits')[0]


#Apex
apex_img = fits.open('../../../data/Apex_ngc6334/13co_apex_J2000_peak_new.fits')[0]


# Define parameters of figure size and panel distribution
figure_xsize = 30.
figure_ysize = 20.
box_xsize = 30.
box_ysize = 10.
right_margin = 1.5
top_margin = 1.5
interplot_xaxis = 0.5
interplot_yaxis = 0.8

fig = plt.figure(figsize=(figure_xsize, figure_ysize))


# Define different sub-panels. You can follow a standard matrix notation:   
#
#   ax11
#   ax21



ax11 = fig.add_axes([(right_margin+0.0*(box_xsize+interplot_xaxis))/figure_xsize, (figure_ysize-(box_ysize+top_margin)-0.0*(box_ysize+interplot_yaxis))/figure_ysize, box_xsize/figure_xsize, box_ysize/figure_ysize], aspect='auto', anchor='SW', projection=wcs.WCS(hdu_image_11.header))

ax21 = fig.add_axes([(right_margin+0.0*(box_xsize+interplot_xaxis))/figure_xsize, (figure_ysize-(box_ysize+top_margin)-1.0*(box_ysize+interplot_yaxis))/figure_ysize, box_xsize/figure_xsize, box_ysize/figure_ysize], aspect='auto', anchor='SW', projection=wcs.WCS(hdu_image_21.header))



#-----------------------------------------------------------------------
# Panel ax11
#
if os.path.isfile('../../../data/Herschel_ngc6334/Doris/NGC6334_Coldens_r18p2_Galactic_delhead.fits') == True:
    my_data = hdu_image_11.data
    my_header = hdu_image_11.header
    image11 = ax11.imshow(my_data, cmap='BuPu', interpolation=None)#vmin=-1.0*0.1*np.nanmax(my_data), vmax=np.nanmax(my_data))
    ra = ax11.coords[0]
    ra.set_major_formatter('hh:mm:ss')
    ra.set_ticks_position('bt')
    dec = ax11.coords[1]
    dec.set_major_formatter('dd:mm:ss')
    dec.set_ticks_position('lr')
    #ax11.coords['ra'].set_axislabel('Right Ascension', fontsize=20)
    #ax11.coords['dec'].set_axislabel('Declination', fontsize=20)
    ax11.tick_params(axis='x', labelsize=12)
    ax11.tick_params(axis='y', labelsize=12)
    
    # Un-comment if you want to remove axis labels and ticks
    ra.set_ticklabel_visible(False)
    ra.set_axislabel('')
    #dec.set_ticklabel_visible(False)
    #dec.set_axislabel('')
    
    # Define center and size of the image
    my_xcenter = Angle(my_header['CRVAL1'], u.degree).degree
    my_ycenter = Angle(my_header['CRVAL2'], u.degree).degree
    my_box = 1500.0/3600.0
    my_xboxmin_pixel_ax11 = my_header['CRPIX1']-1+(my_xcenter-my_box/2.0-my_header['CRVAL1'])/my_header['CDELT1']
    my_xboxmax_pixel_ax11 = my_header['CRPIX1']-1+(my_xcenter+my_box/2.0-my_header['CRVAL1'])/my_header['CDELT1']
    my_yboxmin_pixel_ax11 = my_header['CRPIX2']-1+(my_ycenter-my_box/2.0-my_header['CRVAL2'])/my_header['CDELT2']
    my_yboxmax_pixel_ax11 = my_header['CRPIX2']-1+(my_ycenter+my_box/2.0-my_header['CRVAL2'])/my_header['CDELT2']
    ax11.set_xlim([my_xboxmax_pixel_ax11, my_xboxmin_pixel_ax11])
    ax11.set_ylim([my_yboxmin_pixel_ax11, my_yboxmax_pixel_ax11])
    
    cax = plt.axes([ax11.get_position().corners()[0][0], ax11.get_position().corners()[1][1]+0.02*(ax11.get_position().corners()[1][1]-ax11.get_position().corners()[0][1]), ax11.get_position().corners()[2][0]-ax11.get_position().corners()[0][0], 0.02*(ax11.get_position().corners()[2][0]-ax11.get_position().corners()[0][0])])
    fig.colorbar(image11, cax=cax, orientation='horizontal', ticklocation='top')


    #ax11.contour(apex_img.data, colors='grey', levels=[5, 15, 25], transform=ax11.get_transform(wcs.WCS(apex_img.header)), zorder=1)
    
    scalebar = ScaleBar(1, location='lower left')#ScaleBar(0.3/(1300*4.84813681e-6*3600), "0.3 pc", color='k', linewidth=3.0, fontsize = 22, corner='bottom left')
    ax11.add_artist(scalebar)
    my_text = "HCN - moment 0"
    ax11.text(my_xboxmax_pixel_ax11+0.5*(my_xboxmin_pixel_ax11-my_xboxmax_pixel_ax11), my_yboxmin_pixel_ax11+0.9*(my_yboxmax_pixel_ax11-my_yboxmin_pixel_ax11), my_text, fontsize=24, clip_on=False, horizontalalignment='center', verticalalignment='bottom')
    
else:
    ax11.get_xaxis().set_ticks([])
    ax11.get_yaxis().set_ticks([])


#-----------------------------------------------------------------------
# Panel ax21
#
if os.path.isfile('../../../data/Herschel_ngc6334/Doris/NGC6334_Dust_Temp_r36p3_Galactic_delhead.fits') == True:
    my_data = hdu_image_21.data
    my_header = hdu_image_21.header
    image21 = ax21.imshow(my_data, cmap='BuPu')#, vmin=-1.0*0.1*np.nanmax(my_data), vmax=np.nanmax(my_data))
    ra = ax21.coords[0]
    ra.set_major_formatter('hh:mm:ss')
    ra.set_ticks_position('bt')
    dec = ax21.coords[1]
    dec.set_major_formatter('dd:mm:ss')
    dec.set_ticks_position('lr')
    #ax21.coords['ra'].set_axislabel('Right Ascension', fontsize=20)
    #ax21.coords['dec'].set_axislabel('Declination', fontsize=20)
    ax21.tick_params(axis='x', labelsize=12)
    ax21.tick_params(axis='y', labelsize=12)
    
    # Un-comment if you want to remove axis labels and ticks
    ra.set_ticklabel_visible(False)
    ra.set_axislabel('')
    #dec.set_ticklabel_visible(False)
    #dec.set_axislabel('')
    
    # Define center and size of the image
    my_xcenter = Angle(my_header['CRVAL1'], u.degree).degree
    my_ycenter = Angle(my_header['CRVAL2'], u.degree).degree
    my_box = 150.0/3600.0
    my_xboxmin_pixel_ax21 = my_header['CRPIX1']-1+(my_xcenter-my_box/2.0-my_header['CRVAL1'])/my_header['CDELT1']
    my_xboxmax_pixel_ax21 = my_header['CRPIX1']-1+(my_xcenter+my_box/2.0-my_header['CRVAL1'])/my_header['CDELT1']
    my_yboxmin_pixel_ax21 = my_header['CRPIX2']-1+(my_ycenter-my_box/2.0-my_header['CRVAL2'])/my_header['CDELT2']
    my_yboxmax_pixel_ax21 = my_header['CRPIX2']-1+(my_ycenter+my_box/2.0-my_header['CRVAL2'])/my_header['CDELT2']
    ax21.set_xlim([my_xboxmax_pixel_ax21, my_xboxmin_pixel_ax21])
    ax21.set_ylim([my_yboxmin_pixel_ax21, my_yboxmax_pixel_ax21])
    
    cax = plt.axes([ax21.get_position().corners()[0][0], ax21.get_position().corners()[1][1]+0.02*(ax21.get_position().corners()[1][1]-ax21.get_position().corners()[0][1]), ax21.get_position().corners()[2][0]-ax21.get_position().corners()[0][0], 0.02*(ax21.get_position().corners()[2][0]-ax21.get_position().corners()[0][0])])
    
    fig.colorbar(image21, cax=cax, orientation='horizontal', ticklocation='top')
    ax21.contour(apex_img.data, colors='grey', levels=[5, 15, 25], transform=ax21.get_transform(wcs.WCS(apex_img.header)), zorder=1)
    
    scalebar = ScaleBar(1, location='lower left')#ScaleBar(0.3/(1300*4.84813681e-6*3600), "0.3 pc", color='k', linewidth=3.0, fontsize = 22, corner='bottom left')
    ax21.add_artist(scalebar)
    my_text = "CCH - moment 0"
    ax21.text(my_xboxmax_pixel_ax21+0.5*(my_xboxmin_pixel_ax21-my_xboxmax_pixel_ax21), my_yboxmin_pixel_ax21+0.90*(my_yboxmax_pixel_ax21-my_yboxmin_pixel_ax21), my_text, fontsize=24, clip_on=False, horizontalalignment='center', verticalalignment='bottom')
    
else:
    ax21.get_xaxis().set_ticks([])
    ax21.get_yaxis().set_ticks([])

    

#-----------------------------------------------------------------------
# Hardcopy plot
os.system('mkdir -p plots/')
os.system('rm -rf plots/hershel.pdf')
plt.savefig('plots/hershel.pdf')
os.system('rm -rf plots/hershel.png')
plt.savefig('plots/hershel.png')
plt.close()
    